# **Data Preparation**

In [1]:
import os
import re
import string
import json

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt

import collections
import re, string
import sys
import time
from nltk.corpus import stopwords
from wordcloud import WordCloud
#from mpl_toolkits.basemap import Basemap

from subprocess import check_output
%matplotlib inline

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import max_error
from math import sqrt

from sklearn.metrics import mean_squared_error, max_error, accuracy_score, confusion_matrix, classification_report, roc_auc_score,roc_curve  # classification metrics

In [2]:
df_yelp_business = pd.read_json('../input/yelp-dataset/yelp_academic_dataset_business.json', lines=True)
df_yelp_business.fillna('NA', inplace=True)

print('Final businesses shape: ', df_yelp_business.shape)
df_yelp_business.head()

In [3]:
df_yelp_review_iter = pd.read_json('../input/yelp-dataset/yelp_academic_dataset_review.json', chunksize=100000, lines=True)

In [4]:
df_yelp_review = pd.DataFrame()
i=0
for df in df_yelp_review_iter:
    df = df[df['business_id'].isin(df_yelp_business['business_id'])]
    df_yelp_review = pd.concat([df_yelp_review, df])
    i=i+1
    print(i)
    if i==7: break

In [5]:
df_yelp_business = df_yelp_business[df_yelp_business['business_id'].isin(df_yelp_review['business_id'])]

In [6]:
print('Final businesses shape: ', df_yelp_business.shape)
print('Final review shape: ', df_yelp_review.shape)

In [7]:
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    return text

In [8]:
df_yelp_review['text'] = df_yelp_review['text'].apply(clean_text)

In [9]:
vectorizer_reviews = CountVectorizer(min_df = .01,max_df = .99, tokenizer = WordPunctTokenizer().tokenize)
vectorized_reviews = vectorizer_reviews.fit_transform(df_yelp_review['text'])

In [10]:
print(vectorized_reviews.shape)

In [11]:
' | '.join(vectorizer_reviews.get_feature_names()[:100]) # only the first 100

In [12]:
vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = lambda x: x.split(', '))
vectorized_categories = vectorizer_categories.fit_transform(df_yelp_business['categories'])

In [13]:
print(vectorized_categories.shape)

In [14]:
' | '.join(vectorizer_categories.get_feature_names()[:100]) # only the first 100

In [15]:
%%time
from scipy import sparse
businessxreview = sparse.csr_matrix(pd.get_dummies(df_yelp_review['business_id']).values)

In [16]:
%%time
from scipy import sparse
businessxreview = sparse.csr_matrix(pd.get_dummies(df_yelp_review['business_id']).values)

In [17]:
print('restaurants x categories: \t', vectorized_categories.shape) 
print('restaurants x reviews: \t\t' , businessxreview.shape) 
print('reviews x words: \t\t', vectorized_reviews.shape)

In [18]:
df_yelp_tips = pd.read_json('../input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True)
df_yelp_tips.head()

In [19]:
df_yelp_user_iter = pd.read_json('../input/yelp-dataset/yelp_academic_dataset_user.json', chunksize=100000, lines=True)

In [20]:
df_yelp_user = pd.DataFrame()
i=0
for df in df_yelp_user_iter:
    df = df[df['user_id'].isin(df_yelp_review['user_id'])]
    df_yelp_user = pd.concat([df_yelp_user, df])
    i=i+1
    print(i)
    if i==7: break

In [21]:
print(df_yelp_business.shape)
print(df_yelp_business.columns)
df_yelp_business.head()

In [22]:
print(df_yelp_review.shape)
print(df_yelp_review.columns)
df_yelp_review.head()

In [23]:
print(df_yelp_user.shape)
print(df_yelp_user.columns)
df_yelp_user.head()

In [24]:
print(df_yelp_tips.shape)
print(df_yelp_tips.columns)
df_yelp_tips.head()

## **Handle Missing Data**

In [25]:
# simple data checking - get row and column of dataframe
print('simple data checking of yelp_business: ',df_yelp_business.shape, df_yelp_business.columns)
print('simple data checking of yelp_review: ',df_yelp_review.shape, df_yelp_review.columns)
print('simple data checking of yelp_tips: ',df_yelp_tips.shape, df_yelp_tips.columns)
print('simple data checking of yelp_user: ',df_yelp_user.shape, df_yelp_user.columns)

In [26]:
# check standard missing value - multiple column
missing_data = df_yelp_business.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

In [27]:
# check standard missing value - multiple column
missing_data = df_yelp_review.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

In [28]:
# check standard missing value - multiple column
missing_data = df_yelp_tips.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

In [29]:
# check standard missing value - multiple column
missing_data = df_yelp_user.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

## **Handle Date Column**

In [30]:
# simple data checking - check 1 of the date column
df_yelp_review['date'].head(15)

In [31]:
df_yelp_review["date"].value_counts()

In [32]:
# change to date type & change date format
series_date_in_date = pd.to_datetime(df_yelp_review["date"],errors='raise',dayfirst=False,yearfirst=True,utc=True)
series_date_in_date

In [33]:
# parsing date column
series_date_in_date = pd.to_datetime(df_yelp_review["date"],errors='raise',dayfirst=False,yearfirst=True,utc=True)

In [34]:
series_date_in_date.dt.day

In [35]:
series_date_in_date.dt.month

In [36]:
df_yelp_review["year"] = series_date_in_date.dt.year

# **Exploratory Data Analysis (EDA)**

In [37]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline

## **Overview of reviews**

In [38]:
useful_reviews = len(df_yelp_review[df_yelp_review["useful"]>0])
cool_reviews = len(df_yelp_review[df_yelp_review["cool"]>0])
funny_reviews = len(df_yelp_review[df_yelp_review["funny"]>0])
negative_reviws = len(df_yelp_review[df_yelp_review["stars"]<2])
positive_reviews =len(df_yelp_review[df_yelp_review["stars"]>3])
total_reviews = len(df_yelp_review)

print("Total reviews: {}".format(total_reviews))
print("Useful reviews: {}".format(useful_reviews))
print("Funny reviews: {}".format(funny_reviews))
print("Cool reviews: {}".format(cool_reviews))
print("Total negative reviews: {}".format(negative_reviws))
print("Total positive reviews: {}".format(positive_reviews))

## **Top reviewed business**

In [39]:
top_reviewed = df_yelp_review[df_yelp_review["stars"]>3]
top_reviews_dict ={}

for business_id in top_reviewed["business_id"].values:
    try :
        top_reviews_dict[business_id] =top_reviews_dict[business_id]+1
    except:
        top_reviews_dict[business_id]=1
        
topbusiness = pd.DataFrame.from_dict(data= top_reviews_dict,orient="index")

topbusiness.reset_index(inplace=True)
topbusiness.columns = ['business_id', 'rated']
del(top_reviews_dict)
del(top_reviewed)

In [40]:
top_count= 20
right=pd.DataFrame(df_yelp_business[['business_id',"name","categories"]].values,
                    columns=['business_id',"Business name","categories"])

top_business_data = pd.merge(topbusiness,right=right, how="inner",on='business_id')
top_business_data.sort_values("rated")[::-1][:top_count].plot(x="Business name",y="rated", 
                                                   kind="bar",figsize=(14,6),
                                                   title='Positive reviews').set_ylabel("Total ratings")

del(topbusiness)
del(right)

## **What are the locations of top reviewed businesses**
Generally customers are eager to know what are the locations of best business outlets etc. In this section we will find locations of top best reviwed business locations and show in map.

In [41]:
num_business = 300
business_ids = top_business_data.sort_values("rated")[::-1][:num_business].business_id.values
#len(business_ids)
useful_b = df_yelp_business.loc[df_yelp_business['business_id'].isin(business_ids)]

#len(useful_b)

In [42]:
fig = plt.figure(figsize=(14, 8), edgecolor='w')

m = Basemap(projection='cyl',llcrnrlon= -180, urcrnrlon = 180,
            llcrnrlat = -90, urcrnrlat= 90,resolution='c',
           lat_ts = True)
m.drawcoastlines()
m.fillcontinents(color='#bbdaa4',lake_color='#FFFFFF')
m.drawcountries()
m.drawmapboundary(fill_color='#FFFFFF')

mloc = m(useful_b['latitude'].tolist(),useful_b['longitude'].tolist())
m.scatter(mloc[1],mloc[0],color ='red',lw=3,alpha=0.3,zorder=5)

## **How businesses are getting reviewed over time**
Some businesses try to maintain their overall quality standards and make their customers happy. Lets see which businesses are maintaining their quality standards based on positive reviews.

In [43]:
num_business = 3
business_ids = top_business_data.sort_values("rated")[::-1][:num_business].business_id.values
business_names = top_business_data.sort_values("rated")[::-1][:num_business]["Business name"].values
for i, business_id in enumerate(business_ids):
    useful_b = df_yelp_review.loc[df_yelp_review['business_id'] == business_id]
    useful_b = useful_b.groupby(['year']).size().reset_index(name='counts')
    #print(useful_b.head())
    series = pd.Series(useful_b["counts"].values, index=useful_b["year"].values, name='Review trend')
    axes = series.plot(kind="bar",figsize=(10, 7))
    plt.xlabel('Year', axes=axes)
    plt.ylabel('Total positive reviews', axes=axes)
    plt.title('Review trend of {}'.format(business_names[i]), axes=axes)
    plt.show()

## **Most recent Trending businesses**

In [44]:
top_business = 3
temp = df_yelp_review[["business_id",'year',"stars"]]
five_star_reviews = temp[temp["stars"]>4]
trending_business_reviews = five_star_reviews.groupby(["business_id",'year']).size().reset_index(name='counts')

trending = trending_business_reviews.sort_values(['year','counts'])[::-1][:top_business].business_id.values
for  business_id in trending:
    record = trending_business_reviews.loc[trending_business_reviews['business_id'] == business_id]
    business_name = df_yelp_business.loc[df_yelp_business['business_id'] == business_id].name.values
    series = pd.Series(record["counts"].values, index=record.year.values, name='Trending business')
    axes = series.plot(kind="bar",figsize=(10, 7))
    plt.xlabel('Year', axes=axes)
    plt.ylabel('Total positive reviews', axes=axes)
    plt.title('Review trend of {}'.format(business_name), axes=axes)
    plt.show()
    #len(trending_business_reviews)

## **Categories of top reviewed businesses**

In [45]:
num_cat =10 # to show top 10 catrgories
top_business = 30 # choose categories of top 30 businesses
cat_data = top_business_data.sort_values("rated")[::-1][:top_business]
#cat_data.categories
Categories={}
for cat in cat_data.categories.values:
    all_categories= cat.split(",")
    for x in all_categories:
        try :
            Categories[x] =Categories[x]+1
        except:
            Categories[x]=1
top_categories = pd.DataFrame.from_dict(data= Categories,orient="index")
top_categories.reset_index(inplace=True)
top_categories.columns = ['category', 'occurance']

x_val=top_categories.sort_values("occurance")[::-1][:num_cat].occurance.values
labels=top_categories.sort_values("occurance")[::-1][:num_cat].category.values
series = pd.Series(x_val, index=labels, name='Top business types')
series.plot.pie(figsize=(10, 10),startangle=90)

## **Categories of trending businesses**

In [95]:
num_cat =10 # to show top 10 catrgories
top_business = 40 # choose categories of top 30 businesses
business_ids = trending_business_reviews.sort_values(['year','counts'])[::-1][:top_business].business_id.values
cat_data = top_business_data.loc[top_business_data['business_id'].isin(business_ids)]
#cat_data.categories
Categories={}
for cat in cat_data.categories.values:
    all_categories= cat.split(",")
    for x in all_categories:
        try :
            Categories[x] =Categories[x]+1
        except:
            Categories[x]=1
top_categories = pd.DataFrame.from_dict(data= Categories,orient="index")
top_categories.reset_index(inplace=True)
top_categories.columns = ['category', 'occurance']

x_val=top_categories.sort_values("occurance")[::-1][:num_cat].occurance.values
labels=top_categories.sort_values("occurance")[::-1][:num_cat].category.values
series = pd.Series(x_val, index=labels, name='Trending business types')
series.plot.pie(figsize=(10, 10),startangle=90)

## **Negatively reviewed businesses**

In [47]:
bottom_reviewed = df_yelp_review[df_yelp_review["stars"]<2]
bottom_reviews_dict ={} 

for business_id in bottom_reviewed["business_id"].values:
    try :
        bottom_reviews_dict[business_id] =bottom_reviews_dict[business_id]+1
    except:
        bottom_reviews_dict[business_id]=1
        
bottombusiness = pd.DataFrame.from_dict(data= bottom_reviews_dict,orient="index")

bottombusiness.reset_index(inplace=True)
#bottombusiness.head()
bottombusiness.columns = ['business_id', 'rated']

In [48]:
top_count= 20
right=pd.DataFrame(df_yelp_business[['business_id',"name","categories"]].values,
                    columns=['business_id',"Business name","categories"])

bottom_business_data = pd.merge(bottombusiness,right=right, how="inner",on='business_id')
bottom_business_data.sort_values("rated")[::-1][:top_count].plot(x="Business name",y="rated", 
                                                   kind="bar",figsize=(14,6),
                                                   title='Negative reviews').set_ylabel("Total 1 star ratings")

del(bottom_reviewed)
del(bottom_reviews_dict)
del(bottombusiness)
del(right)

## **Business categories which needs improvement**

In [49]:
num_cat =10 # to show top 10 catrgories
bottom_business = 30 # choose categories of top 30 businesses
cat_data = bottom_business_data.sort_values("rated")[::-1][:bottom_business]

Categories={}
for cat in cat_data.categories.values:
    all_categories= cat.split(",")
    for x in all_categories:
        try :
            Categories[x] =Categories[x]+1
        except:
            Categories[x]=1
bottom_categories = pd.DataFrame.from_dict(data= Categories,orient="index")
bottom_categories.reset_index(inplace=True)
bottom_categories.columns = ['category', 'occurance']

x_val=bottom_categories.sort_values("occurance")[::-1][:num_cat].occurance.values
labels=bottom_categories.sort_values("occurance")[::-1][:num_cat].category.values
series = pd.Series(x_val, index=labels, name='Categories')
series.plot.pie(figsize=(10, 10),startangle=90)

## **Most frequent words in Positive and Negative reviews**

In [50]:
import collections
from wordcloud import WordCloud

In [51]:
# these are helper functions 
# directly copied from https://gist.github.com/benhoyt/dfafeab26d7c02a52ed17b6229f0cb52

def tokenize(s):
    """Convert string to lowercase and split into words (ignoring
    punctuation), returning list of words.
    """
    word_list = re.findall(r'\w+', s.lower())
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    return filtered_words


def count_ngrams(lines, min_length=2, max_length=4):
    """Iterate through given lines iterator (file object or list of
    lines) and return n-gram frequencies. The return value is a dict
    mapping the length of the n-gram to a collections.Counter
    object of n-gram tuple and number of times that n-gram occurred.
    Returned dict includes n-grams of length min_length to max_length.
    """
    lengths = range(min_length, max_length + 1)
    ngrams = {length: collections.Counter() for length in lengths}
    queue = collections.deque(maxlen=max_length)

    # Helper function to add n-grams at start of current queue to dict
    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length:
                ngrams[length][current[:length]] += 1

    # Loop through all lines and words and add n-grams to dict
    for line in lines:
        for word in tokenize(line):
            queue.append(word)
            if len(queue) >= max_length:
                add_queue()

    # Make sure we get the n-grams at the tail end of the queue
    while len(queue) > min_length:
        queue.popleft()
        add_queue()

    return ngrams

def print_most_frequent(ngrams, num=10):
    """Print num most common n-grams of each length in n-grams dict."""
    for n in sorted(ngrams):
        print('----- {} most common {}-word phrase -----'.format(num, n))
        for gram, count in ngrams[n].most_common(num):
            print('{0}: {1}'.format(' '.join(gram), count))
        print('')

def print_word_cloud(ngrams, num=5):
    """Print word cloud image plot """
    words = []
    for n in sorted(ngrams):
        for gram, count in ngrams[n].most_common(num):
            s = ' '.join(gram)
            words.append(s)
            
    cloud = WordCloud(width=1440, height= 1080,max_words= 200).generate(' '.join(words))
    plt.figure(figsize=(20, 15))
    plt.imshow(cloud)
    plt.axis('off');
    plt.show()
    print('')

In [52]:
#positive review
num_business_analysis = 1 # basically this will tell how much computing and diverse our analysis will be
business_ids=  top_business_data.sort_values("rated")[::-1][:num_business_analysis].business_id.values
business_names =   top_business_data.sort_values("rated")[::-1][:num_business_analysis]["Business name"].values
# get all the reviews and analyse them
#business_names
for i, business_id in enumerate(business_ids):
    # now extract reviews from reviews data
    print("Analysing business: ",business_names[i])
    reviews = df_yelp_review.loc[df_yelp_review['business_id'] == business_id].text.values
    most_used_text = count_ngrams(reviews,max_length=3)
    print_most_frequent(most_used_text, num=10)
    print_word_cloud(most_used_text, 10)
    #print ("total reviews ",len(reviews))

In [53]:
#negative review
num_business_analysis = 1 # basically this will tell how much computing and diverse our analysis will be
business_ids=bottom_business_data.sort_values("rated")[::-1][:num_business_analysis].business_id.values
business_names = bottom_business_data.sort_values("rated")[::-1][:num_business_analysis]["Business name"].values
# get all the reviews and analyse them
#business_names
for i, business_id in enumerate(business_ids):
    # now extract reviews from reviews data
    print("Analysing business: ",business_names[i])
    reviews = df_yelp_review.loc[df_yelp_review['business_id'] == business_id].text.values
    most_used_text = count_ngrams(reviews,max_length=3)
    print_most_frequent(most_used_text, num=10)
    print_word_cloud(most_used_text, 10)
    #print ("total reviews ",len(reviews))

## **Most frequent tips**

In [54]:
num_business_analysis = 2 # basically this will tell how much computing and diverse our analysis will be
business_ids=bottom_business_data.sort_values("rated")[::-1][:num_business_analysis].business_id.values
business_names = bottom_business_data.sort_values("rated")[::-1][:num_business_analysis]["Business name"].values
# get all the reviews and analyse them
#business_names
for i, business_id in enumerate(business_ids):
    # now extract reviews from reviews data
    print("Analysing business: ",business_names[i])
    reviews = df_yelp_tips.loc[df_yelp_tips['business_id'] == business_id].text.values
    most_used_text = count_ngrams(reviews,max_length=4)
    print_most_frequent(most_used_text, num=10)
    print_word_cloud(most_used_text, 10)
    #print ("total reviews ",len(reviews))

**Relationship between users's friends and review patterns**

In [55]:
top_users = 15
user_most_reviews = df_yelp_review.groupby(['user_id']).size().reset_index(name='counts')
top_users_ids = user_most_reviews.sort_values(['counts'])[::-1][:top_users].user_id.values

In [56]:
user_frnds = {}
for  users_id in top_users_ids:
    
    record = df_yelp_user.loc[df_yelp_user['user_id'] == users_id]
    user_frnds[users_id] = {}
    user_frnds[users_id]["name"]= record.name.values[0]
    user_frnds[users_id]["friends"]= record.friends.values[0]
    if record.friends is not "None":
        user_frnds[users_id]["friends_count"]= len(record.friends.values[0].split(","))
    else:
        user_frnds[users_id]["friends_count"]=0
    user_frnds[users_id]["review_count"]=record.review_count.values[0]
    user_frnds[users_id]["fans"]=record.fans.values[0]

In [57]:
user_friend_df = pd.DataFrame.from_dict(data= user_frnds,orient="index")

In [58]:
user_friend_df.head()

In [59]:
axes = user_friend_df.plot(x="name", y=["review_count", "friends_count", "fans"],
                           kind="bar",figsize=(12, 7))
plt.xlabel('Name', axes=axes)
plt.ylabel('Count', axes=axes)
plt.title('Top User Review trend'.format(business_name), axes=axes)
plt.show()

**Does user's friends influence business review?**

In [60]:
business_rank = 4 # 0 to 9
temp = df_yelp_review[["business_id",'year',"stars"]]
five_star_reviews = temp[temp["stars"]>4]
trending_business_reviews = five_star_reviews.groupby(["business_id",'year']).size().reset_index(name='counts')

business_id = trending_business_reviews.sort_values(['year','counts'])[::-1][:10].business_id.values[business_rank]
business_name = df_yelp_business.loc[df_yelp_business['business_id'] == business_id].name.values[0]

In [61]:
user_reviws = df_yelp_review.loc[df_yelp_review['business_id']==business_id]
topuser_reviws = user_reviws.groupby(['user_id']).size().reset_index(name='counts')
print("Total users who gave ratings to {} are {}".format(business_name,len(topuser_reviws)))

In [62]:
top_users = 50 #len(topuser_reviws)// 10
topuser_reviws = topuser_reviws.sort_values(['counts'])[::-1][:top_users]

In [63]:
%%time

users_ids = topuser_reviws.user_id.values
users_ids = pd.merge(topuser_reviws,right=df_yelp_user, how="inner",on='user_id')
users_ids =users_ids [["name","user_id", "friends"]]

In [64]:
%%time
def calc_frnd_review(frnds, business_id, df_yelp_review):
        frnds = frnds.split(',')
        # count reviews
        #frnds = yelp_users.loc[(yelp_users['user_id'].isin(frnds)) & (yelp_users["review_count"]>0 )].user_id.values
        friend_review = df_yelp_review.loc[(df_yelp_review['business_id']==business_id) &
                                            (df_yelp_review['user_id'].isin(frnds))
                                           ][["stars","user_id"]]
        friend_review_cnt = len(friend_review.user_id.values)
        if(friend_review_cnt>0):
            total_stars = friend_review.stars.sum()
        else:
            total_stars = 0
        return  friend_review_cnt,total_stars

users_ids[["frnd_count","total_stars"]]= users_ids["friends"].apply(lambda frnds: pd.Series(calc_frnd_review(frnds, business_id, df_yelp_review), index=['frnd_count','total_stars']))

In [65]:
users_ids[users_ids["frnd_count"]>0]

# **Modelling**

## **Predict Rating from Review Text**

In [66]:
# split data
from sklearn.model_selection import train_test_split

# preprocessing and layer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# visualisasi plot
import matplotlib.pyplot as plt

In [67]:
a = df_yelp_business[df_yelp_business['categories'].str.contains('Restaurant') == True]
rev = df_yelp_review[df_yelp_review.business_id.isin(a['business_id']) == True]

In [68]:
rev_samp = rev.sample(n = 350000, random_state = 42)
train = rev_samp[0:280000]
test = rev_samp[280000:]

In [69]:
train.shape, test.shape

In [70]:
#train = pd.read_csv('/home/adam/R/Yelp/dataset/model_train.csv', usecols = ['text', 'stars'])
train = train[['text', 'stars']]
train['stars'].hist();train.head()

In [71]:
train = pd.get_dummies(train, columns = ['stars'])
train.head()

In [72]:
#test = pd.read_csv('/home/adam/R/Yelp/dataset/model_test.csv', usecols=['text', 'stars'])
test = test[['text', 'stars']]
test = pd.get_dummies(test, columns = ['stars'])
train.shape, test.shape

In [73]:
# set frac = 1. to use the entire sample
train_samp = train.sample(frac = .1, random_state = 42)
test_samp = test.sample(frac = .1, random_state = 42)
train_samp.shape, test_samp.shape

In [74]:
# let's see the correlation matrix
mask = np.triu(np.ones_like(rev.corr()))
plt.figure(figsize= (30, 10))
sns.heatmap(rev.corr(), annot=True, cmap= 'PuBuGn', mask= mask)
plt.show()

**Naive Bayes Linear Model**

In [75]:
# max_features is an upper bound on the number of words in the vocabulary
max_features = 5000
tfidf = TfidfVectorizer(max_features = max_features)

In [76]:
class NBFeatures(BaseEstimator):
    '''Class implementation of Jeremy Howards NB Linear model'''
    def __init__(self, alpha):
        # Smoothing Parameter: always going to be one for my use
        self.alpha = alpha
        
    def preprocess_x(self, x, r):
        return x.multiply(r)
    
    # calculate probabilities
    def pr(self, x, y_i, y):
        p = x[y == y_i].sum(0)
        return (p + self.alpha)/((y==y_i).sum()+self.alpha)
    
    # calculate the log ratio and represent as sparse matrix
    # ie fit the nb model
    def fit(self, x, y = None):
        self._r = sparse.csr_matrix(np.log(self.pr(x, 1, y) /self.pr(x, 0, y)))
        return self
    
    # apply the nb fit to original features x
    def transform(self, x):
        x_nb = self.preprocess_x(x, self._r)
        return x_nb

In [77]:
# Create pipeline using sklearn pipeline:
    # I basically create my tfidf features which are fed to my NB model 
    # for probability calculations. Then those are fed as input to my 
    # logistic regression model.
lr = LogisticRegression(random_state=42)
nb = NBFeatures(1)
p = Pipeline([
    ('tfidf', tfidf),
    ('nb', nb),
    ('lr', lr)
])

In [78]:
class_names = ['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']
scores = []
preds = np.zeros((len(test_samp), len(class_names)))
for i, class_name in enumerate(class_names):
    train_target = train_samp[class_name]    
    cv_score = np.mean(cross_val_score(estimator = p, X = train_samp['text'].values, 
                                      y = train_target, cv = 3, scoring = 'accuracy'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))
    p.fit(train_samp['text'].values, train_target)
    preds[:,i] = p.predict_proba(test_samp['text'].values)[:,1]

In [79]:
train['text'][108912]

In [80]:
import statsmodels.api as sm # estimates statistical models
from sklearn.feature_selection import RFE #Recursive Feature Elimination for feature selection
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support as score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [81]:
s = metrics.confusion_matrix(np.argmax(test_samp[class_names].values, axis = 1),np.argmax(preds, axis = 1))
t = metrics.classification_report(np.argmax(test_samp[class_names].values, axis = 1),np.argmax(preds, axis = 1))

print("Test Results")
#pred_train = gnb.predict(X_train)

print(s)
print("\n")
print(t)
print("\n")
#print("MAE test score:", mean_absolute_error(np.argmax(test_samp[class_names].values, axis = 1),np.argmax(preds, axis = 1))
#print("RMSE test score:", sqrt(mean_squared_error(np.argmax(test_samp[class_names].values, axis = 1),np.argmax(preds, axis = 1))))

**Neural Network Model**

In [82]:
# Change the data type to str and numpy array 
text = train_samp['text'].astype(str)
label = train_samp[['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']].values

In [83]:
rating_train, rating_test, label_train, label_test = train_test_split(text, label, test_size = 0.3)

In [84]:
# convert to sequence
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(rating_train) 
tokenizer.fit_on_texts(rating_test)
 
sekuens_train = tokenizer.texts_to_sequences(rating_train)
sekuens_test = tokenizer.texts_to_sequences(rating_test)
 
padded_train = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

In [85]:
from keras.layers.convolutional import Conv1D

In [86]:
model = Sequential([
    Embedding(input_dim=9000, output_dim=16),
    LSTM(64),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

In [87]:
Adam(learning_rate=0.00146, name='Adam')
model.compile(optimizer = 'Adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [88]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.9):
      print("\nThe train and validation accuracy obtained has reached the value of > 90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [89]:
num_epochs = 15
history = model.fit(padded_train, label_train, epochs=num_epochs, validation_data=(padded_test, label_test), verbose=2, callbacks=[callbacks])

In [90]:
print("Train Results")
y_pred=model.predict(padded_train) 
y_pred=np.argmax(y_pred, axis=1)
y_train=np.argmax(label_train, axis=1)

print(confusion_matrix(y_train, y_pred))
print("\n")
print(classification_report(y_train, y_pred))

In [91]:
print("Test Results")
y_pred=model.predict(padded_test) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(label_test, axis=1)

print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

## **Recommendation System**

In [92]:
# to choose a restaurant, just copy the business id and paste it in the next cell
# you can always rerun the cell to choose another restuarant. 
df_yelp_business.sample(10)

In [93]:
import operator
from scipy.spatial.distance import cdist

def business_choose(name):
    new_reviews = df_yelp_review.loc[df_yelp_review['business_id'] == name, 'text']
    
    new_categories = df_yelp_business.loc[df_yelp_business['business_id'] == name, 'categories']
    
    # find most similar reviews
    dists1 = cdist(vectorizer_reviews.transform(new_reviews).todense().mean(axis=0), 
              vectorized_reviews.T.dot(businessxreview).T.todense(), 
               metric='correlation')
    
    # find most similar categories
    dists2 = cdist(vectorizer_categories.transform(new_categories).todense().mean(axis=0), 
              vectorized_categories.todense(), 
               metric='correlation')
    
    # combine the two vectors in one matrix
    dists_together = np.vstack([dists1.ravel(), dists2.ravel()]).T
    
    # this is a key cell: how are we going to prioritize ?
    dists = dists_together.mean(axis=1)
    
    # select the closest 10
    closest = dists.argsort().ravel()[:10]
    
    display('Selected Business => ',df_yelp_business.loc[df_yelp_business['business_id']== name, ['business_id', 'categories', 'name', 'stars']])
    
    display('Recomended Business => ',df_yelp_business.loc[df_yelp_business['business_id'].isin(df_yelp_business['business_id'].iloc[closest]), ['business_id', 'categories', 'name', 'stars']])

In [94]:
business_choose('0agr_FyDdcMQtv-JCLOnhg')